# Define HeatModule enitity

In [1]:
import crestdsl.model as crest
import crestdsl.ui as crestui

In [2]:
# define the required resources
onOff = crest.Resource(unit="onOff", domain=["on", "off"])
watt = crest.Resource(unit="Watt", domain=crest.REAL)  
celsius = crest.Resource(unit="Celsius", domain=crest.REAL)
time = crest.Resource(unit="Time", domain=crest.REAL)

class HeatModule(crest.Entity):
    switch = crest.Input(resource=onOff, value="on")
    electricity = crest.Input(resource=watt, value=0)
    internal_temp = crest.Local(resource=celsius, value=0)
    timer = crest.Local(resource=time, value=0)
    heating = crest.Output(resource=watt, value=0)
    
    # states
    off = current = crest.State()
    on = crest.State()
    error = crest.State()
    
    # transitions
    @crest.transition(source=off, target=on)
    def to_on(self):
        return self.switch.value == "on" and self.timer.value <= 0 and self.electricity.value >= 200
    @crest.transition(source=on, target=off)
    def to_off(self):
        return self.switch.value != "on" or self.timer.value >= 30 or self.electricity.value < 200
    @crest.transition(source=on, target=error)
    def to_error(self):
        return self.internal_temp.value >= 400
    
    # updates for heat energy output
    @crest.update(state=on, target=heating)
    def on_update_output(self, dt):
        # 50 per cent efficiency
        return self.electricity.value * 0.5 
    @crest.update(state=off, target=heating)
    def off_update_output(self, dt):
        return 0
    @crest.update(state=error, target=heating)
    def error_update_output(self, dt):
        return 0
    
    # update timer:
    @crest.update(state=on, target=timer)
    def on_update_timer(self, dt):
        return self.timer.value + dt
    @crest.update(state=off, target=timer)
    def off_update_timer(self, dt):
        new_value = self.timer.value - 2 * dt
        if new_value <= 0:   # don't go below 0
            return 0
        else:
            return new_value

    # updates for internal_temp
    @crest.update(state=on, target=internal_temp)
    def on_update_internal_temp(self, dt):
        # if more than 200 watt, we grow
        # one tenth degree per extra watt per time unit
        # if lower, we sink at the same rate
        factor = (self.electricity.value - 200) / 10
        
        if self.electricity.value >= 200:
            return self.internal_temp.value + factor * dt
        else:
            new_value = self.internal_temp.value + factor * dt
            return max(new_value, 22)  # don't go below 22
        
    @crest.update(state=[off,error], target=internal_temp)
    def off_error_update_internal_temp(self, dt):
        # see formula above
        new_value = self.internal_temp.value - 20 * dt
        return max(new_value, 22)  # don't go below 22


Plot it

In [3]:
crestui.plot(HeatModule())

# Run Standard Simulator

In [4]:
from crestdsl.simulation import Simulator

mod = HeatModule()      # create a crest system
print(mod.current, mod.timer.value, mod.internal_temp.value) 
# output : HeatModule.off 0 0

sim = Simulator(mod)    # instantiate the simulator
sim.stabilise()         # perform the stabilisation routine
print(mod.current, mod.timer.value, mod.internal_temp.value) 
# output: HeatModule.off 0 22

mod.switch.value = "on" # modify the system's input
mod.electricity.value = 300
sim.stabilise()         # stabilise again
print(mod.current, mod.timer.value, mod.internal_temp.value) 
# output: HeatModule.on 0 22

sim.advance(10)         # advance time for 10 time units
print(mod.current, mod.timer.value, mod.internal_temp.value) 
# output: HeatModule.on 10 122.0

HeatModule.off 0 0
HeatModule.off 0 22
HeatModule.on 0 22
HeatModule.on 10 122.0


# Run Interactive Simulator

In [ ]:
from crestdsl.simulation import InteractiveSimulator

module = HeatModule()
print(mod.current, mod.timer.value, mod.internal_temp.value) 
# output : HeatModule.off 0 0

sim = InteractiveSimulator(module)
sim.stabilise()
module.switch.value = "on"
sim.stabilise()
module.electricity.value = 326
sim.stabilise()
print(mod.current, mod.timer.value, mod.internal_temp.value) 

sim.advance(30)
print(mod.current, mod.timer.value, mod.internal_temp.value) 

HeatModule.on 10 122.0
HeatModule.on 10 122.0

 Non-Determinism detected 
There are multiple enabled transitions in entity:  HeatModule 
(Current time: 30 -- Current automaton state: on)

Choose one of the following transitions by entering the according number:
0  ... to_off (transition to 'off')
1  ... to_error (transition to 'error')

Other commands:
r  ... choose a transition randomly
p  ... plot the system
pe ... plot the entity in which non-determinism occurs
q! ... to exit the script (not recommended in Jupyter mode)

Any other input will be interpreted.
This means you can use it to e.g. inspect ports values.
The entity HeatModule is bound to the variable entity.
Example: entity.my_port.value will print the value of port my_port.

